In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
df1 = pd.read_csv("datasets_20710_26737_Bengaluru_House_Data.csv")
df1.head(10)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
6,Super built-up Area,18-May,Old Airport Road,4 BHK,Jaades,2732,4.0,NaN,204.00
7,Super built-up Area,Ready To Move,Rajaji Nagar,4 BHK,Brway G,3300,4.0,NaN,600.00
8,Super built-up Area,Ready To Move,Marathahalli,3 BHK,NaN,1310,3.0,1.0,63.25
9,Plot Area,Ready To Move,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.00


In [4]:
df=df1.copy()

In [5]:
df = df.drop(['area_type','society','balcony','availability'],axis='columns')
df.shape

(13320, 5)

In [6]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [8]:
df2 = df1.drop(['area_type','society','balcony','availability'],axis='columns')
df2.shape

(13320, 5)

In [10]:
df3 = df2.dropna()
df3.isnull().sum()

location      0
size          0
total_sqft    0
bath          0
price         0
dtype: int64

In [11]:
df3['bhk'] = df3['size'].apply(lambda x: int(x.split(' ')[0]))
df3.bhk.unique()

D:\ML\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 27, 10, 19, 16, 43, 14, 12,
       13, 18], dtype=int64)

In [12]:
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        return None

In [13]:
df4 = df3.copy()
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head(2)


,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4


In [14]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000/df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [15]:
df5.location = df5.location.apply(lambda x: x.strip())
location_stats = df5['location'].value_counts(ascending=False)
location_stats

Whitefield         533
Sarjapur  Road     392
Electronic City    304
Kanakpura Road     264
Thanisandra        235
                  ... 
Panathur Road,       1
PC Palaya            1
Tippenahalli         1
Chokkahalli          1
Udayagiri            1
Name: location, Length: 1287, dtype: int64

In [16]:
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

Ganga Nagar             10
Sector 1 HSR Layout     10
Basapura                10
Nagappa Reddy Layout    10
Gunjur Palya            10
                        ..
Panathur Road,           1
PC Palaya                1
Tippenahalli             1
Chokkahalli              1
Udayagiri                1
Name: location, Length: 1047, dtype: int64

In [17]:

df5.location = df5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
len(df5.location.unique())

241

In [18]:

df5[df5.total_sqft/df5.bhk<300].head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
9,other,6 Bedroom,1020.0,6.0,370.0,6,36274.509804
45,HSR Layout,8 Bedroom,600.0,9.0,200.0,8,33333.333333
58,Murugeshpalya,6 Bedroom,1407.0,4.0,150.0,6,10660.980810
68,Devarachikkanahalli,8 Bedroom,1350.0,7.0,85.0,8,6296.296296
70,other,3 Bedroom,500.0,3.0,100.0,3,20000.000000


In [19]:
df6 = df5[~(df5.total_sqft/df5.bhk<300)]
df6.shape

(12456, 7)

In [20]:
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = np.mean(subdf.price_per_sqft)
        st = np.std(subdf.price_per_sqft)
        reduced_df = subdf[(subdf.price_per_sqft>(m-st)) & (subdf.price_per_sqft<=(m+st))]
        df_out = pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out
df7 = remove_pps_outliers(df6)
df7.shape


(10242, 7)

In [21]:

def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df.price_per_sqft),
                'std': np.std(bhk_df.price_per_sqft),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')
df8 = remove_bhk_outliers(df7)
# df8 = df7.copy()
df8.shape

(7317, 7)

In [22]:
df9 = df8[df8.bath<df8.bhk+2]
df9.shape

(7239, 7)

In [23]:
df10 = df9.drop(['size','price_per_sqft'],axis='columns')
df10.head(3)

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3


In [30]:
df=df10.copy()

In [34]:
df.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [35]:
X=df[['location','total_sqft','bath','bhk']]
y=df['price']

In [36]:
from sklearn.preprocessing import LabelEncoder

In [37]:
le=LabelEncoder()
le.fit(X['location'])

LabelEncoder()

In [38]:
X['location']=le.transform(X['location'])

D:\ML\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
X

,location,total_sqft,bath,bhk
0,0,2850.0,4.0,4
1,0,1630.0,3.0,3
2,0,1875.0,2.0,3
3,0,1200.0,2.0,3
4,0,1235.0,2.0,2
...,...,...,...,...
10233,240,1200.0,2.0,2
10234,240,1800.0,1.0,1
10237,240,1353.0,2.0,2
10238,240,812.0,1.0,1


In [40]:
from sklearn.preprocessing import RobustScaler

In [42]:
rb=RobustScaler()

In [43]:
rb.fit(X)

RobustScaler()

In [44]:
X=rb.transform(X)

In [45]:
X

array([[-1.01388889,  2.83555556,  2.        ,  2.        ],
       [-1.01388889,  0.66666667,  1.        ,  1.        ],
       [-1.01388889,  1.10222222,  0.        ,  1.        ],
       ...,
       [ 0.65277778,  0.17422222,  0.        ,  0.        ],
       [ 0.65277778, -0.78755556, -1.        , -1.        ],
       [ 0.65277778,  4.16888889,  3.        ,  2.        ]])

In [46]:
X=pd.DataFrame(X)

In [47]:
X

,0,1,2,3
0,-1.013889,2.835556,2.0,2.0
1,-1.013889,0.666667,1.0,1.0
2,-1.013889,1.102222,0.0,1.0
3,-1.013889,-0.097778,0.0,1.0
4,-1.013889,-0.035556,0.0,0.0
...,...,...,...,...
7234,0.652778,-0.097778,0.0,0.0
7235,0.652778,0.968889,-1.0,-1.0
7236,0.652778,0.174222,0.0,0.0
7237,0.652778,-0.787556,-1.0,-1.0


In [48]:
from sklearn.model_selection import train_test_split

In [50]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [56]:
from sklearn.linear_model import RidgeCV


In [57]:
lr1=RidgeCV()
lr1.fit(X_train,y_train)
lr1.score(X_test,y_test)

0.7229972274502383

In [58]:
pickle.dump(lr1,open('lr1.pkl','wb'))
pickle.dump(le,open('le1.pkl','wb'))
pickle.dump(rb,open('rb.pkl','wb'))

In [59]:
model=pickle.load(open('lr1.pkl','rb'))
le=pickle.load(open('le1.pkl','rb'))
rb=pickle.load(open('rb.pkl','rb'))

In [60]:
list1=[['1st Block Jayanagar',2000,2,2]]

In [62]:
list2=pd.DataFrame(list1)

In [66]:
list2[0]= le.transform(list2[0])

In [68]:
list2=rb.transform(list2)

In [69]:
list2

array([[-1.01388889,  1.32444444,  0.        ,  0.        ]])

In [70]:
list2=pd.DataFrame(list2)

In [71]:
result=model.predict(list2)

In [72]:
result

array([132.38505138])

In [73]:
a='1st Block Jayanagar'
b=1000

In [74]:
list3=[[a,b]]

In [75]:
list3

[['1st Block Jayanagar', 1000]]